In [10]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows", None)

import json
from bs4 import BeautifulSoup
!pip install folium
import folium
!pip install geopy
from geopy.geocoders import Nominatim





In [11]:
import requests

In [12]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [14]:
soup = BeautifulSoup(data, 'html.parser')

In [15]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [25]:
for row in soup.find("table").find_all('tr'):
    cells=row.find_all('td')
    if(len(cells)>0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text) 

In [27]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList, "Borough" : boroughList, "Neighborhood" : neighborhoodList})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


NameError: name 'toronto_df_dropna' is not defined

In [30]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront
3,M6A\n,North York\n,Lawrence Manor / Lawrence Heights
4,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government


In [31]:
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge\n"
1,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G\n,Scarborough\n,"Woburn, Woburn\n"
4,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae\n"


In [32]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned\n":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge\n"
1,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G\n,Scarborough\n,"Woburn, Woburn\n"
4,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae\n"


In [35]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G\n", "M2H\n", "M4B\n", "M1J\n", "M4G\n", "M4M\n", "M1R\n", "M9V\n", "M9L\n", "M5V\n", "M1B\n", "M5A\n"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G\n,Downtown Toronto\n,"Central Bay Street, Central Bay Street\n"
1,M2H\n,North York\n,"Hillcrest Village, Hillcrest Village\n"
2,M4B\n,East York\n,"Parkview Hill / Woodbine Gardens, Parkview Hil..."
3,M1J\n,Scarborough\n,"Scarborough Village, Scarborough Village\n"
4,M4G\n,East York\n,"Leaside, Leaside\n"
5,M4M\n,East Toronto\n,"Studio District, Studio District\n"
6,M1R\n,Scarborough\n,"Wexford / Maryvale, Wexford / Maryvale\n"
7,M9V\n,Etobicoke\n,South Steeles / Silverstone / Humbergate / Jam...
8,M9L\n,North York\n,"Humber Summit, Humber Summit\n"
9,M5V\n,Downtown Toronto\n,CN Tower / King and Spadina / Railway Lands / ...


In [36]:
toronto_df_grouped.shape

(103, 3)